In [0]:
import tensorflow as tf
tf.set_random_seed(42)

In [114]:
tf.__version__

'1.15.0-rc3'

In [0]:
import keras

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### 1. Read the dataset

In [0]:
import pandas as pd

data = pd.read_csv('/content/drive/My Drive/bank.csv')

In [120]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [122]:
data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
RowNumber,10000,NaN,NaN,NaN,5000.5,2886.9,1,2500.75,5000.5,7500.25,10000
CustomerId,10000,NaN,NaN,NaN,1.56909e+07,71936.2,1.55657e+07,1.56285e+07,1.56907e+07,1.57532e+07,1.58157e+07
Surname,10000,2932,Smith,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CreditScore,10000,NaN,NaN,NaN,650.529,96.6533,350,584,652,718,850
Geography,10000,3,France,5014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gender,10000,2,Male,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,10000,NaN,NaN,NaN,38.9218,10.4878,18,32,37,44,92
Tenure,10000,NaN,NaN,NaN,5.0128,2.89217,0,3,5,7,10
Balance,10000,NaN,NaN,NaN,76485.9,62397.4,0,0,97198.5,127644,250898
NumOfProducts,10000,NaN,NaN,NaN,1.5302,0.581654,1,1,1,2,4


In [123]:
data.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

#### 2.Drop the columns which are unique for all users like IDs

In [124]:
data.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### 3.Distinguish the feature and target set

In [0]:
X=data.drop(['Exited'],axis=1)
y = data['Exited']

In [126]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
dtypes: float64(2), int64(6), object(2)
memory usage: 781.3+ KB


In [127]:
!pip install category_encoders

In [0]:
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
le = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
data_cat = le.fit_transform(X)

#### 4.Divide the data set into Train and test sets

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [130]:
display('X_train',X_train.shape,'X_test',X_test.shape,'y_train',y_train.shape,'y_test',y_test.shape)

'X_train'

(8000, 10)

'X_test'

(2000, 10)

'y_train'

(8000,)

'y_test'

(2000,)

In [131]:
X_train_cat=le.transform(X_train)
X_test_cat=le.transform(X_test)
#X_train_cat=X_train_cat.to_numpy()
display(X_train_cat.head(),X_test_cat.head())

,CreditScore,Geography_France,Geography_Spain,Geography_Germany,Gender_Female,Gender_Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
9254,686,1,0,0,0,1,32,6,0.00,2,1,1,179093.26
1561,632,0,0,1,0,1,42,4,119624.60,2,1,1,195978.86
1670,559,0,1,0,0,1,24,3,114739.92,1,1,0,85891.02
6087,561,1,0,0,1,0,27,9,135637.00,1,1,0,153080.40
6669,517,1,0,0,0,1,56,9,142147.32,1,0,0,39488.04


,CreditScore,Geography_France,Geography_Spain,Geography_Germany,Gender_Female,Gender_Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
6252,596,0,0,1,0,1,32,3,96709.07,2,0,0,41788.37
4684,623,1,0,0,0,1,43,1,0.00,2,1,1,146379.30
1731,601,0,1,0,1,0,44,4,0.00,2,1,0,58561.31
4742,506,0,0,1,0,1,59,8,119152.10,2,1,1,170679.74
4521,560,0,1,0,1,0,27,7,124995.98,1,1,1,114669.79


In [135]:
y_train.head()

9254    0
1561    0
1670    1
6087    1
6669    1
Name: Exited, dtype: int64

In [136]:
X_train_stan

array([[ 0.35649971,  1.00150113, -0.57638802, ...,  0.64920267,
         0.97481699,  1.36766974],
       [-0.20389777, -0.99850112, -0.57638802, ...,  0.64920267,
         0.97481699,  1.6612541 ],
       [-0.96147213, -0.99850112,  1.73494238, ...,  0.64920267,
        -1.02583358, -0.25280688],
       ...,
       [ 0.86500853,  1.00150113, -0.57638802, ..., -1.54035103,
        -1.02583358, -0.1427649 ],
       [ 0.15932282,  1.00150113, -0.57638802, ...,  0.64920267,
        -1.02583358, -0.05082558],
       [ 0.47065475, -0.99850112, -0.57638802, ...,  0.64920267,
         0.97481699, -0.81456811]])

#### 4.Normalize the train and test data

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train_stan = sc.fit_transform(X_train_cat)
X_test_stan = sc.transform(X_test_cat)

In [0]:
from keras.models import Sequential
from keras.layers import Dense

#### 6. Initialize & build the model
#### 7. Optimize the model 

In [0]:
# Initializing the ANN
model = Sequential()

In [0]:
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 13, units=6, kernel_initializer='uniform'))

In [0]:
# Adding the second hidden layer
# Notice that we do not need to specify input dim. 
classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform'))

In [0]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

In [0]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [156]:
classifier.fit(X_train_stan,y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 2s 200us/step - loss: 0.4912 - acc: 0.7942
Epoch 2/100
8000/8000 [==============================] - 1s 127us/step - loss: 0.4336 - acc: 0.7945
Epoch 3/100
8000/8000 [==============================] - 1s 137us/step - loss: 0.4267 - acc: 0.8061
Epoch 4/100
8000/8000 [==============================] - 1s 135us/step - loss: 0.4195 - acc: 0.8291
Epoch 5/100
8000/8000 [==============================] - 1s 136us/step - loss: 0.4135 - acc: 0.8306
Epoch 6/100
8000/8000 [==============================] - 1s 137us/step - loss: 0.4096 - acc: 0.8345
Epoch 7/100
8000/8000 [==============================] - 1s 139us/step - loss: 0.4063 - acc: 0.8356
Epoch 8/100
8000/8000 [==============================] - 1s 135us/step - loss: 0.4044 - acc: 0.8345
Epoch 9/100
8000/8000 [==============================] - 1s 135us/step - loss: 0.4021 - acc: 0.8339
Epoch 10/100
8000/8000 [==============================] - 1s 133us/step - loss: 0.4015 - acc: 0.8370

#### 8.Predict the results using 0.5 as a threshold

In [169]:
#Predicting the Test set results
y_pred = classifier.predict(X_test_stan)
#Threshold of 50%
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [False]])

#### 9.Print the Accuracy score and confusion matrix 

In [170]:
#Making the COnfusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1531,   76],
       [ 196,  197]])

#### We can observation that the model accurately predicted (1531 + 197) correctl and (76+196) incorrectly with 86% accuracy